In [1]:
pip install qiskit==1.1.0 qiskit-aer==0.13.3 qiskit-algorithms==0.3.0 qiskit-optimization==0.6.1 numpy requests


Note: you may need to restart the kernel to use updated packages.


In [5]:
# ============================================================
# QAOA TSP SOLVER (VS CODE, modern Qiskit 1.x version)
# Uses data from QuCO-CSAM GitHub repository
# ============================================================

import numpy as np  # For numerical operations and array handling
import requests     # To download files from GitHub if not present locally
import os           # To check for file existence

from qiskit_algorithms import QAOA  # Quantum Approximate Optimization Algorithm
from qiskit_algorithms.optimizers import SPSA  # Stochastic optimizer for quantum parameters
from qiskit.primitives import BackendSampler  # For executing circuits on backends
from qiskit_aer import Aer  # High-performance quantum simulator backend

from qiskit_optimization.applications import Tsp  # TSP problem wrapper for Qiskit
from qiskit_optimization.algorithms import MinimumEigenOptimizer  # Uses QAOA to solve optimization problems

# ============================================================
# DOWNLOAD REQUIRED FILES FROM GITHUB (IF NOT PRESENT)
# ============================================================

# Base URL where the required TSP data files are hosted
BASE_URL = (
    "https://raw.githubusercontent.com/QuCO-CSAM/"
    "Solving-Combinatorial-Optimisation-Problems-Using-Quantum-Algorithms/"
    "main/TravellingSalesmanProblem/Data/"
)

FILES = ["Matrices.txt", "optimal.txt"]  # List of files to download


# Function to download a file if it does not exist locally
def download_if_missing(filename):
    if not os.path.exists(filename):  # Check if file exists
        print(f"Downloading {filename} ...")
        response = requests.get(BASE_URL + filename)  # Request file from GitHub
        response.raise_for_status()  # Raise error if download fails
        with open(filename, "w") as f:  # Write file locally
            f.write(response.text)
        print(f"{filename} downloaded.")
    else:
        print(f"{filename} already exists.")  # Skip download if file already present

# Loop over all required files
for f in FILES:
    download_if_missing(f)  # Ensure all necessary data files are present

# ============================================================
# LOAD DISTANCE MATRICES
# ============================================================

def readInData():
    G = []  # Will store distance matrices for all TSP sizes
    sizes = [3, 4, 5, 6, 7, 8, 9, 10, 11]  # City counts
    counts = [n * n for n in sizes]  # Number of entries in each matrix

    # Open and read the raw matrix data
    with open("Matrices.txt", "r") as f:
        raw = f.read().split()  # Split file into list of numbers

    values = [int(float(x)) for x in raw]  # Convert strings to integers

    idx = 0
    for n, cnt in zip(sizes, counts):  # Loop over each matrix size
        block = values[idx:idx + cnt]  # Extract the relevant segment
        G.append(np.reshape(block, (n, n)))  # Reshape into n x n matrix
        idx += cnt  # Move index forward

    return G  # Return list of matrices

# ============================================================
# LOAD WARM-START PARAMETERS
# ============================================================

def optimal(a, b, c, f_len, u, filename="optimal.txt"):
    with open(filename, "r") as file:
        tokens = file.read().split()  # Read all numbers in file

    vals = []
    for t in tokens:
        if t != ",":  # Skip commas
            vals.append(float(t[:-1]))  # Remove trailing characters and convert to float


#precomputed classical parameter vectors used as initial points (warm starts) for the QAOA algorithm.
# They are not random, and they are not quantum states.
# Instead, they are:
# Classical real-valued numbers
# Representing angles used in parameterized quantum gates
# Learned or computed offline (classically) and stored in optimal.txt

#one variable for each of the five TSP problem sizes (3 to 8 cities) and larger

    # Split warm-start vectors according to precomputed lengths
    v = np.array(vals[0:a])
    r = np.array(vals[a:a + b])
    o = np.array(vals[a + b:a + b + c])
    d = np.array(vals[a + b + c:a + b + c + f_len])
    z = np.array(vals[a + b + c + f_len:a + b + c + f_len + u])

    return [v, r, o, d, z]  # Return as list of arrays

# ============================================================
# FINAL QAOA TSP SOLVER
# ============================================================

def quantumApproximateOptimizationAlgorithm(
    numIter,
    numShots,
    distance_matrix,
    pValue,
    deviceName="aer_simulator",
    initialPoint=None,
):
    tsp_app = Tsp(distance_matrix)  # Wrap distance matrix into TSP problem object
    qp = tsp_app.to_quadratic_program()  # TO QUBO, Convert TSP to a quadratic program suitable for QAOA

    #WHAT DOES TO QUADRATIC_PROGRAM DO INTERNALLY?
    # It formulates the TSP as a QUBO problem by:
    # Defining binary variables for city visits
    # Creating a cost function based on the distance matrix
    # Adding constraints to ensure valid tours

    # Choose simulator or quantum backend
    backend = Aer.get_backend(deviceName)  
    sampler = BackendSampler(backend, options={"shots": numShots})  # Wrap backend for QAOA execution

    optimizer = SPSA(maxiter=numIter)  # Classical optimizer for quantum parameters

    # Instantiate QAOA with backend, optimizer, circuit depth, and optional warm-start
    qaoa = QAOA(
        sampler=sampler,
        optimizer=optimizer,
        reps=pValue,
        initial_point=initialPoint,
    )

#Inside solve() Qiskit does:
# Converts QP → QUBO
# Converts QUBO → Ising Hamiltonian
# then builds:
# Cost Hamiltonian
# Mixer Hamiltonian
# Constructs the QAOA circuit
#all done by Qiskit internally

    #find the minimum eigenvalue of the Hamiltonian corresponding to the QP.
    solver = MinimumEigenOptimizer(qaoa)  # Wrap QAOA as optimizer for quadratic program
    #SPSA optimizer iteratively calls the QAOA circuit to minimize the cost function.
    result = solver.solve(qp)  # Solve the TSP

    route = tsp_app.interpret(result)  # Convert result to city route
    length = tsp_app.tsp_value(route, distance_matrix)  # Compute total distance

    mes = result.min_eigen_solver_result  # Access internal solver details
    opt_time = getattr(mes, "optimizer_time", None)  # Extract optimizer execution time if available

    return length, opt_time, route, result  # Return tour length, time, route, and raw result

# ============================================================
# MAIN EXECUTION
# ============================================================


#CODE FOR FEASIBLE SOLUTION!!!!!


if __name__ == "__main__":
    print("\nLoading data...")
    distance_matrices = readInData()  # Load all distance matrices
    R = optimal(54, 96, 100, 216, 294)  # Load warm-start vectors

    distance_matrix = distance_matrices[1]  # Select 3-city TSP
    numIter = 1  # Number of optimizer iterations
    numShots = 8192  # Number of measurements per circuit execution
    pValue = 3  # Circuit depth / QAOA layers
    initialPoint = R[1][: 2 * pValue]  # Warm-start vector for QAOA

    print("\nRunning QAOA...")
    length, time_taken, route, raw = quantumApproximateOptimizationAlgorithm(
        numIter,
        numShots,
        distance_matrix,
        pValue,
        "aer_simulator",
        initialPoint,
    )

    print("\nRESULTS")
    print("Route:", route)  # Sequence of cities to visit
    print("Tour length:", length)  # Total distance of touro
    print("Optimizer time:", time_taken)  # Time taken by classical optimizer


# ============================================================
# DEBUGGING EXECUTION BLOCK (5x5 TSP)
# ============================================================

# if __name__ == "__main__":
#     print("\n--- DEBUGGING DATA LOADING ---")
#     distance_matrices = readInData()
#     R = optimal(54, 96, 100, 216, 294)

#     # 1. Check Matrix Size (Index 2 should be 5x5)
#     distance_matrix = distance_matrices[2] 
#     n_cities = len(distance_matrix)
#     print(f"Matrix selected. Problem size: {n_cities} cities.")
#     print(f"Required qubits (N^2): {n_cities**2}")

#     # 2. Check Parameter Length
#     pValue = 3  # Number of layers
#     required_params = 2 * pValue  # QAOA needs 2 parameters per layer
    
#     # Selecting the 5x5 warm-start array (R[2]) and slicing it
#     available_params = len(R[2])
#     initialPoint = R[2][:required_params]
    
#     print(f"QAOA Layers (p): {pValue}")
#     print(f"Parameters required by circuit: {required_params}")
#     print(f"Parameters available in R[2]: {available_params}")
#     print(f"Parameters being passed: {len(initialPoint)}")

#     # Safety check: If we don't have enough values, the code will crash here.
#     if len(initialPoint) < required_params:
#         print("CRITICAL ERROR: initialPoint is too short for pValue!")
#     else:
#         print("\n--- RUNNING QAOA ---")
#         # Increase iterations for 5x5. numIter=1 is often insufficient for convergence.
#         numIter = 50 
#         numShots = 8192

#         try:
#             length, time_taken, route, raw = quantumApproximateOptimizationAlgorithm(
#                 numIter,
#                 numShots,
#                 distance_matrix,
#                 pValue,
#                 "aer_simulator",
#                 initialPoint,
#             )

#             print("\n--- RESULTS ---")
#             print("Route found:", route)
#             print("Tour length:", length)
#             print("Optimizer time:", time_taken)
            
#             # Check if result is a valid permutation (every city visited once)
#             if sorted(route) == list(range(n_cities)):
#                 print("SUCCESS: Solution is feasible.")
#             else:
#                 print("WARNING: Solution is INFEASIBLE (invalid TSP tour).")

#         except Exception as e:
#             print(f"\nAN ERROR OCCURRED: {e}")


Matrices.txt already exists.
optimal.txt already exists.

Loading data...

Running QAOA...

RESULTS
Route: [0, 2, 1, 3]
Tour length: 1318.0
Optimizer time: 6.868483066558838


Brazil 4x4 Matrix

In [ ]:

import numpy as np
import os
from qiskit_algorithms import QAOA
from qiskit_algorithms.optimizers import SPSA
from qiskit.primitives import BackendSampler
from qiskit_aer import Aer
from qiskit_optimization.applications import Tsp
from qiskit_optimization.algorithms import MinimumEigenOptimizer


def setup_files():
    # We create the tsp4x4.txt file locally with your specific matrix
    matrix_data = """
    0.    653.  926.  2264.
    653.  0.    1025. 2861.
    926.  1025. 0.    2885.
    2264. 2861. 2885. 0.
    """
    with open("tsp4x4.txt", "w") as f:
        f.write(matrix_data)
    print("tsp4x4.txt created locally.")
#========================================================

def read4x4Data():
    # This version specifically loads one 4x4 matrix
    with open("tsp4x4.txt", "r") as f:
        raw = f.read().split()
    
    values = [float(x) for x in raw]
    # Reshape the 16 numbers into a 4x4 matrix
    return np.reshape(values, (4, 4))

# ============================================================

def load_optimal_params(pValue, filename="optimal.txt"):
    # This logic is more robust for parsing tokens
    if not os.path.exists(filename):
        print(f"Warning: {filename} not found. Using random initial point.")
        return None
        
    with open(filename, "r") as file:
        raw_data = file.read().replace(',', ' ').split()
    
    vals = []
    for t in raw_data:
        try:
            # Try to clean trailing commas or characters
            clean_t = t.rstrip(',')
            vals.append(float(clean_t))
        except ValueError:
            continue

    # For 4x4 (Size index 1 in the original R array), 
    # we need 2 * pValue parameters
    # Here we take a segment that corresponds to 4x4 parameters
    # Original 'r' vector was indices 54 to 54+96
    start_idx = 54
    params = np.array(vals[start_idx : start_idx + (2 * pValue)])
    return params

# ============================================================

def run_qaoa_tsp(distance_matrix, pValue, numIter=1, numShots=8192):
    tsp_app = Tsp(distance_matrix)
    qp = tsp_app.to_quadratic_program()

    backend = Aer.get_backend("aer_simulator")
    sampler = BackendSampler(backend, options={"shots": numShots})
    optimizer = SPSA(maxiter=numIter)

    # Load parameters specifically for 4x4
    initial_pt = load_optimal_params(pValue)

    qaoa = QAOA(
        sampler=sampler,
        optimizer=optimizer,
        reps=pValue,
        initial_point=initial_pt,
    )

    solver = MinimumEigenOptimizer(qaoa)
    result = solver.solve(qp)

    route = tsp_app.interpret(result)
    length = tsp_app.tsp_value(route, distance_matrix)
    
    return length, route

# ============================================================

if __name__ == "__main__":
    setup_files() # Ensure the 4x4 file exists
    
    print("Loading 4x4 Matrix...")
    dist_matrix = read4x4Data()
    
    pValue = 3 # Circuit depth
    
    print(f"Running QAOA for 4x4 TSP (Qubits required: {4**2})...")
    
    # Increase iterations slightly (10-20) if you want better accuracy than numIter=1
    length, route = run_qaoa_tsp(dist_matrix, pValue, numIter=10)

    print("\n" + "="*30)
    print("FINAL RESULTS")
    print("="*30)
    print(f"Optimal Route: {route}")
    print(f"Total Distance: {length}")
    print("="*30)

tsp4x4.txt created locally.
Loading 4x4 Matrix...
Running QAOA for 4x4 TSP (Qubits required: 16)...

FINAL RESULTS
Optimal Route: [2, 3, 0, 1]
Total Distance: 6827.0
